In [7]:
print(torch.backends.mps.is_available())

True


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to(0)

text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt").to(0)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


model-00001-of-00019.safetensors:   0%|             | 0.00/4.89G [00:00<?, ?B/s]
model-00001-of-00019.safetensors:   0%|    | 10.5M/4.89G [00:01<10:55, 7.45MB/s]
model-00001-of-00019.safetensors:   0%|    | 21.0M/4.89G [00:03<13:02, 6.22MB/s]
model-00001-of-00019.safetensors:   1%|    | 31.5M/4.89G [00:04<12:27, 6.51MB/s]
model-00001-of-00019.safetensors:   1%|    | 41.9M/4.89G [00:06<11:49, 6.84MB/s]
model-00001-of-00019.safetensors:   1%|    | 52.4M/4.89G [00:07<11:45, 6.86MB/s]
model-00001-of-00019.safetensors:   1%|    | 62.9M/4.89G [00:09<11:05, 7.26MB/s]
model-00001-of-00019.safetensors:   2%|    | 73.4M/4.89G [00:11<13:35, 5.91MB/s]
model-00001-of-00019.safetensors:   2%|    | 83.9M/4.89G [00:13<13:55, 5.76MB/s]
model-00001-of-00019.safetensors:   2%|    | 94.4M/4.89G [00:14<13:19, 6.00MB/s]
model-00001-of-00019.safetensors:   2%|     | 105M/4.89G [00:16<13:21, 5.97MB/s]
model-00001-of-00019.safetensors:   2%|     | 115M/4.89G [00:18<13:28, 5.91MB/s]
model-00001-of-00019.safeten

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map='mps')

text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt").to("mps")

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

RuntimeError: No GPU found. A GPU is needed for quantization.

In [4]:
!pip install -U bitsandbytes

DEPRECATION: Loading egg at /Users/aswin/miniconda3/envs/mlenv/lib/python3.11/site-packages/huggingface_hub-0.20.1-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.7 MB/s eta 0:00:0000:0100:01


In [9]:
import transformers
transformers.__version__

'4.37.2'